In [1]:
#lib para facilitar o uso de dataframes
import pandas as pd
#lib para facilitar alguns cálculos
import numpy as np

In [2]:
#lendo o csv com o dataset pronto
df_voting = pd.read_csv('data/cleaned_voting.csv')

In [4]:
#Confirmando o formato dos dados
df_voting.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,n,y,n,y,y,n,n,y,y,n,n,y
1,republican,y,y,n,y,y,y,n,n,n,y,n,y,y,y,n,y
2,democrat,y,y,y,n,n,n,y,y,y,n,n,n,n,n,y,y
3,democrat,n,y,n,y,y,y,n,n,n,n,y,y,n,y,n,n
4,republican,n,n,n,y,y,y,y,n,n,y,n,y,y,y,n,y


# Regras de associacao

Regras de associacao sao tecnicas de mineiracao de dados usadas para descobrir associacoes interessantes entre atributos de um determinado banco de dados.
A definicao classica de regras de associacao foi apresentada em Agrawal, Imielinski, & Swami (1993) and Han & Kamber, (2006)), e e definida como o seguinte:

Seja T {t1,t2...tn} um conjunto de transacoes e I {i1,i2...in} um conjunto de itens, definimos D como o dado relevante para a tarefa sendo um conjunto de transacoes onde cada T e um conjunto de itens de maneira que T seja um subconjunto de I.

Uma regra de associacao tem a forma X -> Y, onde X e Y sao conjuntos disjuntos

Em nosso dataset as transacoes T representao o conjunto de votos de cada representante, e os Itens representao a reposta ao voto para cada projeto.

O algoritmo aprior é usado para encontrar conjuntos frequentes de items em databases como o nosso.
Ele funciona com a afirmativa que:
> Um sub conjunto de um conjunto frequente provavelmente e trambem um conjunto frequente

Por exemplo:
   se {i1,i2} é um conjunto frequente entao provavelmente {i1} e {i2} tambem devem ser conjuntos de items frequentes
   
- O algoritmo faz varias iteracoes em busca de itens frequentes e usa esses itens frequentes para gerar as regras de associacao

Abaixo vemos a implementacao do algoritmo passo a passo